# Focused Coding

In [38]:
# import packages
import pandas as pd 
import os
from tqdm import tqdm
from nltk.tokenize import TweetTokenizer
import nltk

from preprocessing_functions import *

In [57]:
df_2 = pd.read_csv('comments.csv', index_col=0)
df_2

,video_id,published_at,like_count,text,author
0,uW6fi2tCnAc,2023-02-19T21:22:45Z,1,"The answer is if China and India don't help, i...",@ilandgrl
1,uW6fi2tCnAc,2023-02-19T00:43:40Z,2,"and that guy is an expert, we're screwed",@lobusdiMortis
2,uW6fi2tCnAc,2023-02-18T22:57:38Z,4,Kennedy is a gem.,@skitz1337
3,uW6fi2tCnAc,2023-02-18T22:22:49Z,0,And just how do we get a nation like China to ...,@dodieodie498
4,uW6fi2tCnAc,2023-02-18T21:44:49Z,3,That man was going for an Oscar,@karenh4458
...,...,...,...,...,...
96590,eAnVFKndFoY,2023-01-23T02:59:10Z,0,"Of course it’s theatre, global warming is a ho...",@internetexplorer6097
96591,eAnVFKndFoY,2023-01-23T02:57:43Z,4,Yup!,@bonitabeach3127
96592,eAnVFKndFoY,2023-01-23T02:57:12Z,5,Staged,@kevinford6372
96593,eAnVFKndFoY,2023-01-23T02:55:36Z,1,Greta Thunberg is a waste of skin and oxygen.....,@stevenewberry6460


In [46]:
df3 = df_2.groupby('author').size().reset_index(name='count')
df3.sort_values(by='count', ascending=False, inplace=True)
df3


,author,count
17606,17605,206
3744,3743,188
18120,18119,166
17605,17604,155
18381,18380,154
...,...,...
21536,21535,1
21537,21536,1
21541,21540,1
21544,21543,1


In [55]:
# process data with using functions from functions.py
processed_df = (
    df_2.pipe(remove_users, 'text')
      .pipe(lowercase_text, 'text')
      .pipe(remove_whitespace, 'text')
      .pipe(remove_punctuation, 'text')
)

In [56]:
processed_df['text'] = processed_df['text'].astype('str')
processed_df.dtypes

video_id        object
published_at    object
like_count       int64
text            object
author          object
dtype: object

In [58]:
extracted_col = df_2["text"]
 
# Add the extracted column to the second DataFrame
processed_df = pd.concat([processed_df, extracted_col.rename("og_text")], axis=1)

In [59]:
processed_df

,video_id,published_at,like_count,text,author,og_text
0,uW6fi2tCnAc,2023-02-19T21:22:45Z,1,the answer is if china and india dont help it ...,@ilandgrl,"The answer is if China and India don't help, i..."
1,uW6fi2tCnAc,2023-02-19T00:43:40Z,2,and that guy is an expert were screwed,@lobusdiMortis,"and that guy is an expert, we're screwed"
2,uW6fi2tCnAc,2023-02-18T22:57:38Z,4,kennedy is a gem,@skitz1337,Kennedy is a gem.
3,uW6fi2tCnAc,2023-02-18T22:22:49Z,0,and just how do we get a nation like china to ...,@dodieodie498,And just how do we get a nation like China to ...
4,uW6fi2tCnAc,2023-02-18T21:44:49Z,3,that man was going for an oscar,@karenh4458,That man was going for an Oscar
...,...,...,...,...,...,...
96590,eAnVFKndFoY,2023-01-23T02:59:10Z,0,of course it’s theatre global warming is a hoa...,@internetexplorer6097,"Of course it’s theatre, global warming is a ho..."
96591,eAnVFKndFoY,2023-01-23T02:57:43Z,4,yup,@bonitabeach3127,Yup!
96592,eAnVFKndFoY,2023-01-23T02:57:12Z,5,staged,@kevinford6372,Staged
96593,eAnVFKndFoY,2023-01-23T02:55:36Z,1,greta thunberg is a waste of skin and oxygen,@stevenewberry6460,Greta Thunberg is a waste of skin and oxygen.....


In [60]:
# use stemming to reduce words to their root words
processed_df = stem_words(processed_df, 'text')

KeyboardInterrupt: 

In [62]:
# use lemmatization to reduce words to their root form
processed_df = lemmatize_words(processed_df, 'text')

In [63]:
processed_df.lemmatized_text = processed_df.lemmatized_text.apply(lambda x: '' if str(x) == 'nan' else x)

In [75]:
substrings = ['god plan', 'greenhouse gas', 'natural cycle', 'hoax']
string = ['cool down']
pattern = '|'.join(string)

In [76]:
filtered_df = processed_df[processed_df['lemmatized_text'].str.contains(pattern, case=False, na=False)]
pd.set_option('display.max_colwidth', None)
print(len(filtered_df))
filtered_df.sample(n=10)

48


video_id          published_at  like_count  \
14811  7LVSrTZDopM  2023-03-04T18:11:28Z           0   
6545   mhDrCPfKgG4  2024-03-25T14:44:06Z           0   
59262  l0bxbks2Izg  2023-04-23T19:47:03Z           2   
62314  a8R-X0kccHo  2023-11-28T11:46:27Z           0   
90205  ThTkXT06UiM  2024-01-19T11:57:14Z           0   
154    703pPQ1o1mY  2023-08-11T23:43:46Z           0   
43228  H2JZ8eI-Yus  2023-02-24T22:27:23Z          52   
62218  a8R-X0kccHo  2023-12-16T21:31:50Z           1   
2566   ry-bRYhN1Xs  2023-02-04T06:52:44Z           0   
5378   mhDrCPfKgG4  2024-03-26T23:29:18Z           0   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

# Add on

In [15]:
tokenizer = TweetTokenizer()

tqdm.pandas() #Creates a progress bar and below use "progress_apply" instead of "apply" to create a progress bar (This is more of a "nice to have" than a "need to have")

#Tokenizing and creating a column of unigrams from the stemmed tweet text. 
df['unigrams'] = df['stemmed_text'].progress_apply(lambda x: tokenizer.tokenize(x))

#Defining a function that will create bigrams 
def bigrams(doc): # a doc is a list of unigrams in same order as in tweets 
    
    bigrams = [] #Empty list to save the bigrams
    
    for bigram in list(nltk.bigrams(doc)):  #Creating bigrams as tuples with nltk.bigrams and iterating over these them
        bigrams.append("_".join(bigram))    #Joining each bigram-tuple pair with an underscore and saving to list
    
    return bigrams

#Creating a column with bigrams by applying function to column of unigrams
df['bigrams'] = df.unigrams.progress_apply(lambda x: bigrams(x))

100%|██████████| 96595/96595 [00:00<00:00, 160833.26it/s]


In [46]:
filtered_df = df[df['lemmatized_text'].str.contains('natural cycle', case=False, na=False)]

In [48]:
len(filtered_df)

60